In [2]:
import pandas as pd
from pathlib import Path

# --- 경로 설정 ---
ROOT_DIR = Path.cwd().parent 
DATA_DIR = ROOT_DIR / "BIgcontest_Data"
CROLLING_DIR = ROOT_DIR / "Crolling"
EDA_DIR = ROOT_DIR / "EDA"
MODEL_DIR = ROOT_DIR / "Model"

In [3]:
import pandas as pd

# CSV 파일을 읽어옵니다.
# 파일이 스크립트와 같은 폴더에 있는지 확인해주세요.
# 한글이 포함된 파일이므로 'cp949' 인코딩을 사용합니다.
try:
    df = pd.read_csv(DATA_DIR/'서울시 성동구 일반음식점 인허가 정보.csv', encoding='cp949')

    # '영업상태명' 컬럼의 값이 '폐업'이 아닌 데이터만 남깁니다.
    df_open = df[df['영업상태명'] != '폐업'].copy()

    # 필터링된 데이터를 새로운 CSV 파일로 저장합니다.
    # 엑셀에서 한글이 깨지지 않도록 'utf-8-sig' 인코딩을 사용합니다.
    output_file_path = DATA_DIR/'서울시_성동구_일반음식점_영업중.csv'
    df_open.to_csv(output_file_path, index=False, encoding='utf-8-sig')

    print(f"작업 완료! '{output_file_path}' 파일로 저장되었습니다.")
    print(f"처리 전 데이터 수: {len(df)}")
    print(f"처리 후 (영업중) 데이터 수: {len(df_open)}")

except FileNotFoundError:
    print("오류: '서울시 성동구 일반음식점 인허가 정보.csv' 파일을 찾을 수 없습니다.")
    print("파일이 올바른 경로에 있는지 확인해주세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

작업 완료! 'c:\Users\hyunj\Bigcontest\BIgcontest_Data\서울시_성동구_일반음식점_영업중.csv' 파일로 저장되었습니다.
처리 전 데이터 수: 14785
처리 후 (영업중) 데이터 수: 3982


In [ ]:
def map_restaurants(open_restaurants_file, big_data_file, output_file):
    """
    두 데이터셋에서 마스킹된 이름과 실제 이름을 비교하여 가맹점을 매핑하고 결과를 저장합니다.

    Args:
        open_restaurants_file (str): 영업 중인 음식점 정보 CSV 파일 경로
        big_data_file (str): 마스킹된 가맹점 이름이 포함된 빅데이터 CSV 파일 경로
        output_file (str): 결과를 저장할 CSV 파일 경로
    """
    try:
        # 데이터 파일들을 읽어옵니다.
        # '서울시_성동구_일반음식점_영업중.csv'는 utf-8-sig로 저장했으므로 그대로 읽습니다.
        restaurants_open = pd.read_csv(open_restaurants_file, encoding='utf-8-sig')
        # big_data_file을 'cp949' 인코딩으로 읽도록 수정합니다.
        big_data = pd.read_csv(big_data_file, encoding='cp949')

        # '사업장명'과 'MCT_NM' 컬럼이 비어있는 경우를 대비하고, 문자열로 타입을 통일합니다.
        restaurants_open['사업장명'] = restaurants_open['사업장명'].astype(str).fillna('')
        big_data['MCT_NM'] = big_data['MCT_NM'].astype(str).fillna('')

        # 매핑된 결과를 저장할 리스트를 생성합니다.
        mapped_data = []

        # 빅데이터 파일의 각 가맹점을 하나씩 확인합니다.
        for index, row in big_data.iterrows():
            masked_name = row['MCT_NM']

            # 요청하신 조건: 이름이 4글자이고 '**'로 끝나는 경우 (예: "가나**")
            if len(masked_name) == 4 and masked_name.endswith('**'):
                prefix = masked_name[:2] # 앞 두 글자를 추출합니다. (예: "가나")

                # 영업 중인 음식점 목록에서 조건에 맞는 후보를 찾습니다.
                potential_matches = restaurants_open[
                    (restaurants_open['사업장명'].str.len() == 4) &
                    (restaurants_open['사업장명'].str.startswith(prefix))
                ]

                # 조건에 맞는 음식점을 찾았다면,
                if not potential_matches.empty:
                    matched_row = potential_matches.iloc[0]
                    mapped_data.append({
                        'ENCODED_MCT': row['ENCODED_MCT'],
                        '사업장명': matched_row['사업장명'],
                        '도로명주소': matched_row['도로명주소']
                    })

        # 매핑된 결과 리스트를 데이터프레임으로 변환합니다.
        mapped_df = pd.DataFrame(mapped_data)

        # 완성된 데이터프레임을 CSV 파일로 저장합니다.
        mapped_df.to_csv(output_file, index=False, encoding='utf-8-sig')

        print(f"총 {len(mapped_df)}개의 가맹점을 매핑하여 '{output_file}' 파일로 저장했습니다.")
        print("\n매핑된 데이터 샘플:")
        print(mapped_df.head())

    except FileNotFoundError as e:
        print(f"오류: {e.filename} 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    except KeyError as e:
        print(f"오류: 파일에서 {e} 컬럼을 찾을 수 없습니다. 컬럼명을 확인해주세요.")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

# --- 아래 파일 이름들을 실제 파일 경로에 맞게 수정하여 사용하세요 ---
open_restaurants_filename = DATA_DIR/'서울시_성동구_일반음식점_영업중.csv'
big_data_filename = DATA_DIR/'big_data_set1_f.csv'
output_filename = DATA_DIR/'mapped_restaurants.csv'

# 매핑 함수를 실행합니다.
map_restaurants(open_restaurants_filename, big_data_filename, output_filename)

In [4]:
import pandas as pd
import requests
import os
import time
from dotenv import load_dotenv

def find_restaurant_name_kakao(address, masked_name, api_key):
    """
    카카오맵 '음식점' 카테고리 내에서 마스킹된 이름과 글자 수가 일치하는 가게를 찾습니다.
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    
    prefix = masked_name.split('*')[0]
    original_len = len(masked_name)
    
    query = f"{address} {prefix}"
    params = {"query": query, "category_group_code": "FD6", "size": 15}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data['documents']:
            for doc in data['documents']:
                place_name = doc['place_name']
                if place_name.startswith(prefix) and len(place_name) == original_len:
                    return place_name
        
        return "Not Found"
        
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {e}")
        return "API Error"
    except Exception as e:
        print(f"처리 중 오류: {e}")
        return "Processing Error"

# --- 메인 코드 실행 부분 ---

# 1. .env 파일에서 환경 변수(API 키) 불러오기
load_dotenv()
kakao_api_key = os.getenv("KAKAO_API_KEY")

if not kakao_api_key:
    print("오류: .env 파일에서 KAKAO_API_KEY를 찾을 수 없거나 파일이 존재하지 않습니다.")
    print("스크립트와 같은 폴더에 .env 파일이 있는지, 파일 내용이 올바른지 확인해주세요.")
else:
    # 2. 데이터 파일 불러오기
    big_data_filename = DATA_DIR/'big_data_set1_f.csv'
    output_filename = DATA_DIR/'big_data_with_kakao_names_v3.csv'

    try:
        big_data = pd.read_csv(big_data_filename, encoding='cp949')
        
        # 테스트를 위해 상위 20개 데이터만 사용 (전체는 .head(20) 제거)
        df_sample = big_data.copy()
        
        df_sample['FOUND_MCT_NM'] = ''

        print("개선된 로직(.env 사용)으로 가맹점 실제 이름을 찾는 중입니다...")
        
        # 3. 각 행에 대해 개선된 함수 실행
        for index, row in df_sample.iterrows():
            if pd.isna(row['MCT_BSE_AR']) or pd.isna(row['MCT_NM']):
                continue
            
            found_name = find_restaurant_name_kakao(row['MCT_BSE_AR'], row['MCT_NM'], kakao_api_key)
            df_sample.loc[index, 'FOUND_MCT_NM'] = found_name
            
            print(f"  - 주소: {row['MCT_BSE_AR']}, 이름: {row['MCT_NM']} -> 찾은 이름: {found_name}")
            time.sleep(0.1)

        # 4. 결과 저장
        df_sample.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"\n작업 완료! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n결과 미리보기:")
        print(df_sample[['MCT_BSE_AR', 'MCT_NM', 'FOUND_MCT_NM']].head())

    except FileNotFoundError:
        print(f"오류: '{big_data_filename}' 파일을 찾을 수 없습니다.")
    except KeyError as e:
        # 'MCT_BSE_AR' 컬럼명을 잘못 입력했을 때를 대비한 오류 처리
        print(f"오류: DataFrame에서 '{e}' 컬럼을 찾을 수 없습니다. CSV 파일의 컬럼명을 확인해주세요.")
    except Exception as e:
        print(f"파일 처리 중 오류가 발생했습니다: {e}")

개선된 로직(.env 사용)으로 가맹점 실제 이름을 찾는 중입니다...
  - 주소: 서울 성동구 마장동, 이름: 성우** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 마장동, 이름: 대보** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 마장동, 이름: 대용** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 마장동, 이름: 통일** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 마장동, 이름: 한울** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 마장동, 이름: 대광** -> 찾은 이름: 대광상회
  - 주소: 서울 성동구 마장동, 이름: 미림** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 마장동, 이름: 호남* -> 찾은 이름: 호남집
  - 주소: 서울 성동구 사근동, 이름: 고향*** -> 찾은 이름: Not Found
  - 주소: 서울 성동구 송정동, 이름: 북* -> 찾은 이름: Not Found
  - 주소: 서울 성동구 용답동, 이름: 좋은** -> 찾은 이름: 좋은세상
  - 주소: 서울 성동구 용답동, 이름: 별난** -> 찾은 이름: 별난만두
  - 주소: 서울 성동구 행당동, 이름: 행당** -> 찾은 이름: 행당족발
  - 주소: 서울 성동구 행당동, 이름: 이혜******* -> 찾은 이름: Not Found
  - 주소: 서울 성동구 홍익동, 이름: 정* -> 찾은 이름: Not Found
  - 주소: 서울 성동구 홍익동, 이름: 조은** -> 찾은 이름: 조은식당
  - 주소: 서울 성동구 행당1동, 이름: 왕십**** -> 찾은 이름: Not Found
  - 주소: 서울  성동구  도선동, 이름: 광한****** -> 찾은 이름: Not Found
  - 주소: 서울  성동구  마장동, 이름: 멕시***** -> 찾은 이름: Not Found
  - 주소: 서울  성동구  마장동, 이름: 도미******** -> 찾은 이름

In [ ]:
import sys
import subprocess
import warnings

# ==============================================================================
# -1. 필수 라이브러리 설치 (실행 환경 초기화 대비)
# ==============================================================================
try:
    print("--- 필수 라이브러리 설치 시작 ---")
    python_executable = sys.executable
    subprocess.check_call([
        python_executable, "-m", "pip", "install",
        "pandas", "requests", "python-dotenv", "tqdm",
        "selenium", "webdriver-manager"
    ])
    print("✅ 라이브러리 설치 완료!")
except Exception as e:
    print(f"❌ 오류: 라이브러리 설치에 실패했습니다: {e}")
    sys.exit(1)

# Selenium 관련 경고 숨기기 (기능 영향 없음)
warnings.filterwarnings("ignore", category=UserWarning, module="selenium")


import pandas as pd
import time
import re
import os
from dotenv import load_dotenv
from tqdm import tqdm

# Selenium 관련 라이브러리
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

def find_name_with_selenium(driver, address, masked_name):
    """
    셀레니움을 이용해 카카오맵에서 검색하고, 조건에 맞는 가게 이름을 찾아 반환합니다.
    """
    try:
        prefix = masked_name.split('*')[0]
        original_len = len(masked_name)
        
        # 주소 마지막 부분 + 가게 이름 앞부분으로 검색어 생성
        last_word_of_address = address.split()[-1] if address.split() else ''
        search_keyword = f"{last_word_of_address} {prefix}"

        # 1. 검색창에 키워드 입력 및 검색
        search_box = driver.find_element(By.ID, "search.keyword.query")
        search_box.clear() # 이전 검색어 지우기
        search_box.send_keys(search_keyword)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2) # 검색 결과 로딩 대기

        # 2. '장소' 탭 클릭 (필요 시)
        try:
            driver.find_element(By.ID, "info.search.place.more").click()
            time.sleep(1)
        except NoSuchElementException:
            pass # 장소 탭이 이미 선택되어 있거나, 버튼이 없는 경우

        # 3. 검색 결과 목록에서 이름 스크래핑
        places = driver.find_elements(By.CSS_SELECTOR, "#info\\.search\\.place\\.list > li")
        
        for place in places:
            try:
                place_name = place.find_element(By.CSS_SELECTOR, "a.link_name").text
                # 조건: 앞부분 일치 & 전체 글자 수 일치
                if place_name.startswith(prefix) and len(place_name) == original_len:
                    return place_name # 조건에 맞는 첫 번째 결과를 반환
            except NoSuchElementException:
                continue # 광고 등 다른 li는 건너뛰기

        return "Not Found" # 조건에 맞는 가게를 못 찾음

    except Exception as e:
        # print(f"  - Selenium 검색 중 오류 발생: {e}")
        return "Selenium Error"

# --- 메인 코드 실행 부분 ---

# 1. 파일 경로 설정
load_dotenv()

# 만약 .env 파일 등을 사용하지 않는 경우, 아래처럼 직접 파일명을 입력하세요.
input_filename = DATA_DIR/'big_data_with_kakao_names_v3.csv'
output_filename = DATA_DIR/'big_data_with_selenium_results.csv'


# 2. Selenium WebDriver 설정
print("Selenium WebDriver를 설정합니다...")
options = webdriver.ChromeOptions()
# options.add_argument("--headless") # 브라우저 창 숨기기
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
print("WebDriver 설정 완료.")

try:
    # 3. 카카오맵 접속
    driver.get("https://map.kakao.com/")
    driver.implicitly_wait(5)

    # 4. 데이터 불러오기 및 'Not Found' 필터링
    df = pd.read_csv(input_filename, encoding='utf-8-sig')
    not_found_df = df[df['FOUND_MCT_NM'] == 'Not Found'].copy()

    if not_found_df.empty:
        print("'Not Found'인 항목이 없습니다. 작업을 종료합니다.")
    else:
        print(f"총 {len(df)}개 데이터 중 'Not Found'인 {len(not_found_df)}개에 대해 Selenium 검색을 시작합니다.")
        
        # 5. 'Not Found' 항목에 대해 Selenium 검색 실행
        for index, row in tqdm(not_found_df.iterrows(), total=not_found_df.shape[0]):
            original_address = row['MCT_BSE_AR']
            masked_name = row['MCT_NM']
            
            if pd.isna(original_address) or pd.isna(masked_name):
                continue
            
            cleaned_address = re.sub(r'\s+[\d-]+$', '', str(original_address)).strip()
            
            # 셀레니움 함수 호출
            found_name = find_name_with_selenium(driver, cleaned_address, masked_name)
            
            # 원본 DataFrame에 결과 업데이트
            df.loc[index, 'FOUND_MCT_NM'] = found_name
        
        # 6. 최종 결과 저장
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"\n작업 완료! 최종 결과가 '{output_filename}' 파일로 저장되었습니다.")
        
        final_found_count = df[~df['FOUND_MCT_NM'].isin(['Not Found', 'Selenium Error'])].shape[0]
        newly_found_count = final_found_count - (len(df) - len(not_found_df))
        
        print(f" - 이번 Selenium 작업을 통해 {newly_found_count}개의 이름을 추가로 찾았습니다.")
        print(f" - 최종적으로 찾은 이름 수: {final_found_count} / {len(df)}")

except FileNotFoundError:
    print(f"오류: 이전 결과 파일인 '{input_filename}'을 찾을 수 없습니다.")
except Exception as e:
    print(f"메인 코드 실행 중 오류가 발생했습니다: {e}")
finally:
    # 7. 드라이버 종료
    print("브라우저를 닫습니다.")
    driver.quit()